### Pick Lottery Numbers

In [7]:
#Inputs

#Define how many times you want to play the lottery
n=584000000

#Choose a lottery number 
myNum=random.sample(range(1, 69), 5)+[random.randint(1,26)]

#Choose whether you would like to multiply winnings by 2. Choosing this usually adds an additional fee per ticket
powerplay=0

In [8]:
print("Your selected lottery numbers:",myNum)
print("Your selected powerball number:",myNum[5])

Your selected lottery numbers: [19, 34, 29, 26, 41, 5]
Your selected powerball number: 5


### Lottery winnings calculation

In [ ]:
import random

In [116]:
def lotteryDraw (selection):

    #lottery draw  
    lottery_draw=random.sample(range(1, 69), 5)
    powerball=random.randint(1,26)
    #lottery_draw=[27, 46, 15, 31, 42]
    #powerball=11
    #print("lottery numbers:",lottery_draw)
    #print("powerball number:",powerball)
    
    #Match your Num
    powerflag=0
    count=0
    powerpick=selection[5]
    
    if powerpick ==  powerball:
        powerflag=1
    
    #go through first 5 numbers and match
    for i in range(0,len(selection)-1):
        #print(i,selection[i])
        if selection[i] in lottery_draw:
            count=count+1
    
    return (powerflag,count)


In [114]:
def lotteryWinnings(powerplay,powerflag, count):
    winnings=0
    wins=0
    
    if powerplay==0:
        if powerflag==1:
            wins=wins+1
            if count==5:
                print ("JACKPOT!!!!!")
                winnings=142600000
            elif count==4:
                winnings=50000
            elif count==3:
                winnings=100
            elif count == 2:
                winnings=7
            else:
                winnings=4
        else:
            if count>2:
                wins=wins+1
                if count==5:
                    winnings=100000
                elif count==4:
                    winnings=100
                else:
                    winnings=7
    else:
        if powerflag==1:
            wins=wins+1
            if count==5:
                print ("JACKPOT!!!!!")
                winnings=142600000
            elif count==4:
                winnings=150000
            elif count==3:
                winnings=300
            elif count == 2:
                winnings=21
            else:
                winnings=12
        else:
            if count>2:
                wins=wins+1
                if count==5:
                    winnings=2000000
                elif count==4:
                    winnings=300
                else:
                    winnings=21
    
    return(winnings,wins)

#### Simulation Function

In [117]:
def simulation(powerplay,n,selection):
    total_winnings=0
    total_wins=0
    
    for i in range(0,n):
        (powerflag, count)=lotteryDraw(selection)
        (winnings, wins)=lotteryWinnings(powerplay,powerflag,count)
        
        total_winnings=total_winnings + winnings
        total_wins=total_wins + wins
        
    return(total_winnings, total_wins)

### Run Simulation

In [174]:
#Simulate winnings
(total_winnings, total_wins)=simulation(powerplay,n,myNum)
spend=n*(2+powerplay)

print("Spend:",spend)
print("Total Winnings:",total_winnings)
print("Total Wins:",total_wins)
print("ROI:", total_winnings/spend)


[59, 24, 57, 40, 42, 13]
JACKPOT!!!!!
JACKPOT!!!!!
Spend: 1168000000
Total Winnings: 431450386
Total Wins: 23532928
ROI: 0.36939245376712326
